# Neuroscience using `fastplotlib` and `pynapple`

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with `pynapple`, to show how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
# if not installed, will need a function from scikit-image
! pip install scikit-image

In [3]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from ipywidgets import IntSlider, Layout, VBox, HBox, FloatSlider
from skimage import measure
from sidecar import Sidecar

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

MESA-INTEL: warning: cannot initialize blitter engine
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


Available devices:
🯄 (default) | Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | Vulkan | Mesa 24.0.8-1
❗ | llvmpipe (LLVM 17.0.6, 256 bits) | CPU | Vulkan | Mesa 24.0.8-1 (LLVM 17.0.6)
❗ | Mesa Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | OpenGL | 


In [4]:
import warnings
warnings.simplefilter('ignore')

## Load the data 

#### Recording of a freely-moving mouse imaged with a Miniscope (1-photon imaging). The area recorded is the postsubiculum - a region that is known to contain head-direction cells, or cells that fire when the animal's head is pointing in a specific direction. 

In [62]:
data = nap.load_file("./data.nwb")

In [63]:
data

data
┍━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                  │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ position_time_support │ IntervalSet │
│ RoiResponseSeries     │ TsdFrame    │
│ calcium_video         │ TsdTensor   │
│ beh_video             │ TsdTensor   │
│ z                     │ Tsd         │
│ y                     │ Tsd         │
│ x                     │ Tsd         │
│ rz                    │ Tsd         │
│ ry                    │ Tsd         │
│ rx                    │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

### Let's view the behavior and calcium data

Hopefully, by the end of the summer we will have developed a tool ([`pynaviz`](https://github.com/pynapple-org/pynaviz)) that makes these visualizations and synchronizations even easier :D

In [7]:
# behavior shape
behavior_data = data["beh_video"]
behavior_data.shape

(9045, 204, 256)

In [8]:
# calcium shape
calcium_data = data["calcium_video"]
calcium_data.shape

(17886, 136, 166)

#### We are going to minimize our view of the data to where both behavior and position data are available:

In [9]:
frame_min = data["position_time_support"]["start"][0]
frame_max = data["position_time_support"]["end"][0]
(frame_min, frame_max)

(7.39305, 1213.22765)

### Create a plot for calcium and behavior video

In [10]:
nap_figure = fpl.Figure(shape=(1,2), names=[["raw", "behavior"]])

nap_figure["raw"].add_image(data=calcium_data[0], name="raw_frame", cmap="viridis")
nap_figure["behavior"].add_image(data=behavior_data[0], cmap="gray")

RFBOutputContext()

Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


<weakproxy at 0x7feec9118c70 to ImageGraphic at 0x7feed7d2cd10>

#### Create a slider that updates behavior and calcium videos so they are aligned

In [11]:
# This time will be in milliseconds
synced_time = FloatSlider(min=frame_min, max=frame_max, step=0.1, description="s", layout=Layout(width="60%"))

def update_time(change):
    # get the index of synced slider
    time_s = change["new"]
    # get the corresponding calcium frame from the pynapple tensor
    frame_raw = calcium_data.get(time_s, time_units="s")
    # update the data in the plot
    nap_figure["raw"].graphics[0].data = frame_raw
    # get the corresponding behavior frame from the pynapple tensor
    frame_behavior = behavior_data.get(time_s, time_units="s")
    # update the data in the plot
    nap_figure["behavior"].graphics[0].data = frame_behavior
    
synced_time.observe(update_time, "value")

In [12]:
sc = Sidecar()
with sc:
    display(VBox([nap_figure.show(), synced_time]))

In [13]:
# manually set the vmin/vmax of the calcium data
nap_figure["raw"]["raw_frame"].vmax = 205
nap_figure["raw"]["raw_frame"].vmin = 25

#### Calculate the contours and overlay them on the raw calcium data

In [14]:
# get the masks
contour_masks = data.nwb.processing['ophys']['ImageSegmentation']['PlaneSegmentation']['image_mask'].data[:]
# reshape the masks into a list of 105 components
contour_masks = list(contour_masks.reshape((len(contour_masks), 166, 136)))

In [15]:
# create a list of the contours
contours = list()
# calculate each contour from the mask
for mask in contour_masks:
    contours.append(np.vstack(measure.find_contours(mask)))

#### Add the calculated contours as an overlay to the calcium video

In [16]:
contours_graphic = nap_figure["raw"].add_line_collection(data=contours, colors="w")

### Select only head-direction neurons

In [17]:
# get the temporal data from the nwb notebook
temporal_data = data["RoiResponseSeries"][:]
temporal_data

Time (s)           0        1        2        3        4  ...
-----------  -------  -------  -------  -------  -------  -----
0.0          0        0.43582  2.96331  0        0        ...
0.033333     0        0.43406  2.95294  0        0        ...
0.066667     0        0.43231  2.9426   0        0        ...
0.1          0        0.43057  2.93231  0        0        ...
0.133333     0        0.42883  2.92205  0        0        ...
0.166667     0        0.4271   2.91182  0        0        ...
0.2          0        0.42537  2.90163  0        0        ...
...
1192.166667  2.54202  0.14531  0.44013  0.5681   0.65477  ...
1192.2       2.53029  0.14775  0.43842  0.56657  0.65227  ...
1192.233333  2.51861  0.14962  0.43671  0.56505  0.64979  ...
1192.266667  2.50698  0.15104  0.435    0.56354  0.64731  ...
1192.3       2.49541  0.15209  0.43331  0.56202  0.64485  ...
1192.333333  2.48389  0.15283  0.43162  0.58476  0.64239  ...
1192.366667  2.47242  0.15333  0.42994  0.62802  0.63994  ...
dt

In [18]:
# compute 1D tuning curved based on head angle
angle = data["ry"]

tuning_curves = nap.compute_1d_tuning_curves_continuous(temporal_data, angle, nb_bins = 120)

In [19]:
# select good components 
good_ixs = list(np.argsort(np.ptp(tuning_curves, axis=0))[-50:])
bad_ixs = list(np.argsort(np.ptp(tuning_curves, axis=0))[:-50])

In [20]:
contours_graphic[good_ixs].colors = "green"
contours_graphic[bad_ixs].colors = "red"

### Remove the "bad" neurons

In [21]:
# sorting neurons based on preferred directions
sorted_ixs = tuning_curves.iloc[:,good_ixs].idxmax().sort_values().index.values

In [22]:
sorted_ixs

array([75, 34, 77, 86, 21, 16,  6,  4, 58, 44, 14, 33, 94, 98, 90, 76,  7,
        5, 82, 28, 15, 88, 45, 39,  0,  8, 20, 13, 24, 60, 18, 27, 10, 78,
        2, 85,  3, 19, 38, 17, 30, 29, 25, 84, 12, 26, 41,  9, 11,  1])

In [23]:
# filter dataset based on sortex indices
temporal_data = temporal_data[:,sorted_ixs]
contours = [contours[i] for i in sorted_ixs]

In [24]:
# only plot the good indices 
nap_figure[0,0].remove_graphic(nap_figure[0,0].graphics[1])
nap_figure[0,0].add_line_collection(data=contours, colors="g")

<weakproxy at 0x7feebd241d00 to LineCollection at 0x7feebc6481d0>

## Make a plot of the calcium traces as a `LineStack`

In [25]:
# create a figure
tstack_fig = fpl.Figure()

RFBOutputContext()

In [26]:
# we need to transpose our temporal data so that it is (# components, time (s))
raw_temporal = temporal_data.to_numpy().T

tstack_graphic = tstack_fig[0,0].add_line_stack(data=raw_temporal, cmap="hsv", name="temporal-stack")

#### Add a `LinearSelector` that we can map to our behavior and calcium videos

In [27]:
tstack_slider = tstack_graphic.add_linear_selector()

In [28]:
tstack_fig.show(maintain_aspect=False)

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

#### Add another function to update the position of the `LinearSelector` based on the position of our `IntSlider` from before

In [29]:
def update_time_temporal(change):
    # get the index of synced slider
    time_s = change["new"]

    # temporal data has framerate 30
    tstack_slider.selection = time_s * 30

synced_time.observe(update_time_temporal, "value")

#### Let's view everything together

In [30]:
VBox([nap_figure.show(), tstack_fig.show(maintain_aspect=False), synced_time])

### Let's make a plot so we can individually view our traces and easily scroll through them

In [33]:
single_temporal = fpl.Figure()

RFBOutputContext()

In [34]:
# select the first component to start
ix = 0
# set the colors of the first component to magenta
tstack_graphic.graphics[ix].colors = "white"
# add the data of the first component as a line
single_temporal[0,0].add_line(data=tstack_graphic.graphics[ix].data.value, colors="white")

<weakproxy at 0x7feebc12f8d0 to LineGraphic at 0x7feebb0e3750>

In [35]:
single_temporal.show(maintain_aspect=False)

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

#### Now, let's add an event handler to update the individual trace based on a selected component

In [22]:
@tstack_fig.renderer.add_event_handler("key_up")
def update_selected_trace(ev):
    global ix
    if ev.key == "ArrowUp":
        # increment ix
        ix += 1
        # check for looping
        if ix == len(tstack_graphic.graphics):
            ix = 0
        # reset the colors to white
        tstack_graphic.colors = "w"
        # update colors of selected component
        tstack_graphic.graphics[ix].colors = "magenta"
        # update single component data
        single_temporal[0,0].graphics[0].data = tstack_graphic.graphics[ix].data.value
        single_temporal[0,0].auto_scale()
    if ev.key == "ArrowDown":
        # decrement ix
        ix -= 1
        # check for looping
        if ix < 0:
            ix = len(temporal_graphic.graphics) - 1
        # reset colors of temporal stack
        tstack_graphic.colors = "w"
        # update colors of selected component
        tstack_graphic.graphics[ix].colors = "magenta"
        # update date of selected component 
        single_temporal[0,0].graphics[0].data = tstack.graphics[ix].data.value
        single_temporal[0,0].auto_scale()

### We can again add a `LinearSelector` and link it to our calcium and behavior data 

In [23]:
single_selector = single_temporal[0,0].graphics[0].add_linear_selector()

In [24]:
def update_selector(change):
    # get the index of synced slider
    time_s = change["new"]

    # temporal data has framerate 30
    single_selector.selection = time_s * 30

In [25]:
synced_time.observe(update_selector, "value")

### Let's look at everything together again


In [26]:
VBox([HBox([nap_figure.show(),VBox([tstack_fig.show(),single_temporal.show()])]), synced_time])

In [ ]:
# tuning curves 
# nemos examples

In [ ]:
# a circle, same color coding as angle of tuning curve 
# arrow that points to current heading 

In [ ]:
# position decoding?

In [93]:
tuning_fig = fpl.Figure()

RFBOutputContext()

In [94]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center

In [95]:
circle = make_circle((0,0), 50, 100)

In [96]:
data["ry"]

Time (s)
----------  -------
7.39305     3.13103
7.4014      3.15806
7.40975     3.1809
7.41805     3.20979
7.4264      3.22044
7.43475     3.2362
7.44305     3.24883
...
1213.17765  5.46986
1213.186    5.45224
1213.19435  5.43446
1213.20265  5.4239
1213.211    5.42556
1213.21935  5.43106
1213.22765  5.44108
dtype: float64, shape: (144709,)

In [97]:
tuning_fig[0,0].add_line(data=circle, cmap="hsv", thickness=10)

<weakproxy at 0x7feead27d8a0 to LineGraphic at 0x7feeafc97a10>

In [98]:
# start rotation 0-45

In [99]:
xs = np.linspace(0, 45, 10)

In [100]:
ys = np.ones((1, 10))

l_data = np.dstack((xs, ys))[0]

In [101]:
line = tuning_fig[0,0].add_line(data=l_data, colors="w", thickness=5)

In [104]:
tuning_fig.show()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [105]:
import pylinalg as la

In [109]:
rot = la.quat_from_euler((0, np.pi/4), order="YZ")

In [110]:
line.rotation = rot

In [111]:
data["ry"].shape

(144709,)

In [135]:
ry_line = np.vstack((data["ry"].t, data["ry"].d)).T

In [205]:
xs = np.linspace(0, 100, 1200)
ys = np.sin(xs)

sine = np.dstack([xs, ys])[0]

In [206]:
angle_fig = fpl.Figure(shape=(1,2))

angle_line = angle_fig[0,0].add_line(sine)

angle_selector = angle_line.add_linear_selector()

angle_fig[0,1].add_line(data=circle, cmap="hsv", thickness=10)
angle_fig[0,1].add_line(data=l_data, colors="w", thickness=5)

RFBOutputContext()

<weakproxy at 0x7feea065bf10 to LineGraphic at 0x7feea0182810>

In [207]:
VBox([nap_figure.show(), angle_fig.show(), synced_time])

In [208]:
def update_line(change):
    t = change["new"]

    angle_selector.selection = t

    ang = data["ry"].get(t)

    rotation = la.quat_from_euler((0, -ang), order="YZ")
    #rotation = lambda theta, hdangle: np.array([[np.cos(theta), np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0],[0,0,1]).dot([np.cos(hdangle), np.sin(hdangle), 0])
    
    

    angle_fig[0,1].graphics[1].rotation = rotation 

In [209]:
synced_time.observe(update_line, "value")